In [1]:
import numpy as np
import pandas as pd
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 소수를 e로 표현하지 않도록 하기
pd.options.display.float_format = "{:.5f}".format

In [2]:
lib_2007 = pd.read_csv("../final_data/lib_2007.csv", index_col=0)
lib_2008 = pd.read_csv("../final_data/lib_2008.csv", index_col=0)
lib_2009 = pd.read_csv("../final_data/lib_2009.csv", index_col=0)
lib_2010 = pd.read_csv("../final_data/lib_2010.csv", index_col=0)
lib_2011 = pd.read_csv("../final_data/lib_2011.csv", index_col=0)
lib_2012 = pd.read_csv("../final_data/lib_2012.csv", index_col=0)
lib_2013 = pd.read_csv("../final_data/lib_2013.csv", index_col=0)
lib_2014 = pd.read_csv("../final_data/lib_2014.csv", index_col=0)
lib_2015 = pd.read_csv("../final_data/lib_2015.csv", index_col=0)
lib_2016 = pd.read_csv("../final_data/lib_2016.csv", index_col=0)
lib_2017 = pd.read_csv("../final_data/lib_2017.csv", index_col=0)
lib_2018 = pd.read_csv("../final_data/lib_2018.csv", index_col=0)
lib_2019 = pd.read_csv("../final_data/lib_2019.csv", index_col=0)
lib_2020 = pd.read_csv("../final_data/lib_2020.csv", index_col=0)
lib_2021 = pd.read_csv("../final_data/lib_2021.csv", index_col=0)

In [3]:
train_lib = pd.concat([lib_2007,lib_2008,lib_2009,lib_2010,lib_2011,lib_2012,lib_2013,lib_2014,lib_2015, lib_2016, lib_2017])
valid_lib = pd.concat([lib_2018, lib_2019])
test_lib = pd.concat([lib_2020,lib_2021])

In [4]:
# train, test set의 데이터 수 구하기
print(f"train set의 데이터 수 : {len(train_lib)}")
print(f"valid set의 데이터 수 : {len(valid_lib)}")
print(f"test set의 데이터 수 : {len(test_lib)}")

train set의 데이터 수 : 9026
valid set의 데이터 수 : 2222
test set의 데이터 수 : 2370


In [5]:
# 사분위수를 활용하여 비율 충족도의 이상치 행 제거하기
def del_outlier(df, col):
    q1 = df[col].quantile(0.10)
    q3 = df[col].quantile(0.90)
    iqr = q3 - q1 
    boundary = 1.5 * iqr 

    upper_index = df[df[col] > q3 + boundary].index
    lower_index = df[df[col] < q1 - boundary].index 

    df.drop(upper_index, inplace = True)
    df.drop(lower_index, inplace = True)

    return df

del_outlier(train_lib, "future_acq_budget_settlement")
del_outlier(valid_lib, "future_acq_budget_settlement")
del_outlier(test_lib, "future_acq_budget_settlement") 

# 이상치 제거 후 train, test set의 데이터 수 구하기
print(f"이상치 제거 후 train set의 데이터 수 : {len(train_lib)}")
print(f"이상치 제거 후 valid set의 데이터 수 : {len(valid_lib)}")
print(f"이상치 제거 후 test set의 데이터 수 : {len(test_lib)}")

이상치 제거 후 train set의 데이터 수 : 8184
이상치 제거 후 valid set의 데이터 수 : 2178
이상치 제거 후 test set의 데이터 수 : 2310


In [6]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max Scaler 생성
scaler = MinMaxScaler()

# 모든 변수에 대해 Min-Max Scaling 적용
# 스케일러를 학습 데이터에 대해 fit
scaler.fit(train_lib.drop(columns=["future_acq_budget_settlement"]))

# 학습 데이터에 대해 변환
train_lib_scaled = pd.DataFrame(scaler.transform(train_lib.drop(columns=["future_acq_budget_settlement"])), 
                                columns=train_lib.columns[:-1])

# 타겟 변수에 대해 스케일링 적용
scaler_y = MinMaxScaler()
train_y_scaled = scaler_y.fit_transform(train_lib[['future_acq_budget_settlement']])

# 학습 데이터에 스케일된 타겟 변수 추가
train_lib_scaled['future_acq_budget_settlement_scaled'] = train_y_scaled

# 검증 데이터에 대해 변환
valid_lib_scaled = pd.DataFrame(scaler.transform(valid_lib.drop(columns=["future_acq_budget_settlement"])), 
                                columns=train_lib.columns[:-1])
valid_y_scaled = scaler_y.transform(valid_lib[['future_acq_budget_settlement']])
valid_lib_scaled['future_acq_budget_settlement_scaled'] = valid_y_scaled

# 테스트 데이터에 대해 변환
test_lib_scaled = pd.DataFrame(scaler.transform(test_lib.drop(columns=["future_acq_budget_settlement"])), 
                               columns=train_lib.columns[:-1])
test_y_scaled = scaler_y.transform(test_lib[['future_acq_budget_settlement']])
test_lib_scaled['future_acq_budget_settlement_scaled'] = test_y_scaled

# X_train, Y_train 구성하기 (스케일된 타겟 사용)
X_train = train_lib_scaled.drop(columns=["future_acq_budget_settlement_scaled"])
Y_train = train_lib_scaled["future_acq_budget_settlement_scaled"]

# X_valid, Y_valid 구성하기 (스케일된 타겟 사용)
X_valid = valid_lib_scaled.drop(columns=["future_acq_budget_settlement_scaled"])
Y_valid = valid_lib_scaled["future_acq_budget_settlement_scaled"]

# X_test, Y_test 구성하기 (스케일된 타겟 사용)
X_test = test_lib_scaled.drop(columns=["future_acq_budget_settlement_scaled"])
Y_test = test_lib_scaled["future_acq_budget_settlement_scaled"]


In [7]:
# columns_to_drop = ["prog_sessions",	"local_mat", "prog_partic", "service_recip", "self_srv_machines", "interlib_loans", 
#                    "reg_members", "borrowers", "user_ed_partic", "web_access", "open_days", 
#                    "avg_week_hours", "disab_mat", "ann_weeding", "support_staff", "info_serv_requests"]

# X_train.drop(columns=columns_to_drop, inplace=True)
# X_valid.drop(columns=columns_to_drop, inplace=True)
# X_test.drop(columns=columns_to_drop, inplace=True)

In [8]:
# 넘파이로 변환
X_train_np = X_train.to_numpy()
X_valid_np = X_valid.to_numpy()
X_test_np = X_test.to_numpy()

# 데이터 형태 맞추기
X_train_reshaped = X_train_np.reshape((X_train_np.shape[0], X_train_np.shape[1], 1))
X_valid_reshaped = X_valid_np.reshape((X_valid_np.shape[0], X_valid_np.shape[1], 1))
X_test_reshaped = X_test_np.reshape((X_test_np.shape[0], X_test_np.shape[1], 1))

In [9]:
import keras
import tensorflow_addons as tfa

inputs=keras.layers.Input(shape=(36,1))
n_classes=1
# Transformer 모델에 들어가는 encoder 첫번째
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    print(x)
    x = keras.layers.Dropout(dropout)(x)
    x1 = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x2 = tfa.layers.InstanceNormalization()(x)
    x = (0.7 * x1) + (0.3 * x2)
    res = x + inputs

    # Feed Forward Part
    x = keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="LeakyReLU")(res)
    x1 = keras.layers.Dropout(dropout)(x)
    x = keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x1 = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x2 = tfa.layers.InstanceNormalization()(x)
    x = (0.7 * x1) + (0.3 * x2)
    return x + res


# Transformer 모델에 들어가는 encoder 두번째
def transformer_encoder2(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)

    x = keras.layers.Dropout(dropout)(x)
    x1 = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x2 = tfa.layers.InstanceNormalization()(x)
    x = (0.7 * x1) + (0.3 * x2)
    res = x + inputs

    # Feed Forward Part
    x = keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="LeakyReLU")(res)
    x1 = keras.layers.Dropout(dropout)(x)
    x = keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x1 = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x2 = tfa.layers.InstanceNormalization()(x)
    x = (0.7 * x1) + (0.3 * x2)
    return x + res


# Transformer 모델 설계 부분
def build_model(
        input_shape,
        head_size,
        num_heads,
        ff_dim,
        num_transformer_blocks,
        mlp_units,
        dropout=0,
        mlp_dropout=0,
):
    print(head_size)
    print(num_heads)
    print(ff_dim)
    inputs = keras.Input(shape=input_shape)
    x = inputs

    for _ in range(num_transformer_blocks):
        x1 = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    print(x)
    for _ in range(num_transformer_blocks):
        x2 = transformer_encoder2(x, head_size, num_heads, ff_dim, dropout)
    print(x2)
    x = (0.5 * x1) + (0.5 * x2)
    x = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = keras.layers.Dense(dim, activation="LeakyReLU")(x)
        x = keras.layers.Dropout(mlp_dropout)(x)
    outputs = keras.layers.Dense(n_classes)(x)
    return keras.Model(inputs, outputs)

2024-09-15 14:05:49.727996: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-15 14:05:49.811005: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 14:05:49.811064: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 14:05:49.811076: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 14:05:49.827674: I tensorflow/core/platform/cpu_feature_g

In [10]:
input_shape = (36,1)
model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=2,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

256
4
4


2024-09-15 14:06:00.519444: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:1a:00.0, compute capability: 7.0


KerasTensor(type_spec=TensorSpec(shape=(None, 36, 1), dtype=tf.float32, name=None), name='multi_head_attention/attention_output/add:0', description="created by layer 'multi_head_attention'")
KerasTensor(type_spec=TensorSpec(shape=(None, 36, 1), dtype=tf.float32, name=None), name='multi_head_attention_1/attention_output/add:0', description="created by layer 'multi_head_attention_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 36, 1), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 36, 1), dtype=tf.float32, name=None), name='tf.__operators__.add_15/Add:0', description="created by layer 'tf.__operators__.add_15'")


In [11]:
from tensorflow.keras.optimizers import Adam
# 모델 컴파일
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics='mse') 

# 모델 학습
history = model.fit(X_train_reshaped, Y_train, epochs=50, batch_size=32, validation_data=(X_valid_reshaped, Y_valid))

Epoch 1/50


2024-09-15 14:06:17.753778: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-09-15 14:06:18.629537: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f90a94f0220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-15 14:06:18.629591: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-09-15 14:06:18.639116: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-15 14:06:18.981797: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


256/256 [==============================] - 33s 25ms/step - loss: 0.0167 - mse: 0.0167 - val_loss: 0.0264 - val_mse: 0.0264
Epoch 2/50
256/256 [==============================] - 6s 22ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.0224 - val_mse: 0.0224
Epoch 3/50
256/256 [==============================] - 6s 22ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.0275 - val_mse: 0.0275
Epoch 4/50
256/256 [==============================] - 5s 21ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.0301 - val_mse: 0.0301
Epoch 5/50
256/256 [==============================] - 6s 24ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.0310 - val_mse: 0.0310
Epoch 6/50
256/256 [==============================] - 6s 22ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.0282 - val_mse: 0.0282
Epoch 7/50
256/256 [==============================] - 6s 24ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.0314 - val_mse: 0.0314
Epoch 8/50
256/256 [==============================] - 6s 22ms/step - loss: 0.00

In [12]:
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from sklearn.metrics import r2_score

# 예측
y_pred = model.predict(X_train_reshaped)

# y_pred와 Y_train를 1차원으로 변환
y_pred = y_pred.flatten()
Y_train = Y_train.to_numpy().flatten()

# MSE
mse = MeanSquaredError()
mse.update_state(Y_train, y_pred)
mse_result = mse.result().numpy()

# RMSE
rmse_result = np.sqrt(mse_result)

# MAE
mae = MeanAbsoluteError()
mae.update_state(Y_train, y_pred)
mae_result = mae.result().numpy()

# SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

smape_result = smape(Y_train, y_pred)

# R² (R Squared)
r2_result = r2_score(Y_train, y_pred)

print(f'MSE: {mse_result}')
print(f'RMSE: {rmse_result}')
print(f'MAE: {mae_result}')
print(f'SMAPE: {smape_result}')
print(f'R²: {r2_result}')

256/256 [==============================] - 3s 7ms/step
MSE: 0.013523926958441734
RMSE: 0.11629241704940796
MAE: 0.0830891877412796
SMAPE: 54.6923261994693
R²: 0.48122261342888906


In [13]:
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from sklearn.metrics import r2_score

# 예측
y_pred = model.predict(X_valid_reshaped)

# y_pred와 Y_valid를 1차원으로 변환
y_pred = y_pred.flatten()
Y_valid = Y_valid.to_numpy().flatten()

# MSE
mse = MeanSquaredError()
mse.update_state(Y_valid, y_pred)
mse_result = mse.result().numpy()

# RMSE
rmse_result = np.sqrt(mse_result)

# MAE
mae = MeanAbsoluteError()
mae.update_state(Y_valid, y_pred)
mae_result = mae.result().numpy()

# SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

smape_result = smape(Y_valid, y_pred)

# R² (R Squared)
r2_result = r2_score(Y_valid, y_pred)

print(f'MSE: {mse_result}')
print(f'RMSE: {rmse_result}')
print(f'MAE: {mae_result}')
print(f'SMAPE: {smape_result}')
print(f'R²: {r2_result}')

69/69 [==============================] - 1s 9ms/step
MSE: 0.01852799579501152
RMSE: 0.1361175775527954
MAE: 0.09319751709699631
SMAPE: 53.72654881969792
R²: 0.32727214644552227


In [14]:
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from sklearn.metrics import r2_score

# 예측
y_pred = model.predict(X_test_reshaped)

# y_pred와 Y_test를 1차원으로 변환
y_pred = y_pred.flatten()
Y_test = Y_test.to_numpy().flatten()

# MSE
mse = MeanSquaredError()
mse.update_state(Y_test, y_pred)
mse_result = mse.result().numpy()

# RMSE
rmse_result = np.sqrt(mse_result)

# MAE
mae = MeanAbsoluteError()
mae.update_state(Y_test, y_pred)
mae_result = mae.result().numpy()

# SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

smape_result = smape(Y_test, y_pred)

# R² (R Squared)
r2_result = r2_score(Y_test, y_pred)

print(f'MSE: {mse_result}')
print(f'RMSE: {rmse_result}')
print(f'MAE: {mae_result}')
print(f'SMAPE: {smape_result}')
print(f'R²: {r2_result}')

73/73 [==============================] - 1s 7ms/step
MSE: 0.014893919229507446
RMSE: 0.12204064428806305
MAE: 0.08466027677059174
SMAPE: 49.84999332261666
R²: 0.4402434736253543


In [28]:
import shap
import matplotlib.pyplot as plt

# 샘플링할 비율 또는 샘플 수 설정
sample_size = 100  # 예를 들어 100개의 샘플을 사용

# 배경 데이터 샘플링
X_train_background = X_train_np[np.random.choice(X_train_np.shape[0], sample_size, replace=False)]

# 예측 데이터 샘플링
X_train_sample = X_train_np[np.random.choice(X_train_np.shape[0], sample_size, replace=False)]

# SHAP explainer 객체 생성
explainer = shap.KernelExplainer(model, X_train_background)

# SHAP 값 계산
shap_values = explainer.shap_values(X_train_sample)

# Summary plot
shap.summary_plot(shap_values, X_train_sample)
plt.show()

  0%|          | 0/100 [00:00<?, ?it/s]2024-09-15 14:47:52.083781: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 29.11GiB (rounded to 31260672000)requested by op Einsum
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-09-15 14:47:52.083850: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-09-15 14:47:52.083865: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 182, Chunks in use: 182. 45.5KiB allocated for chunks. 45.5KiB in use in bin. 1.0KiB client-requested in use in bin.
2024-09-15 14:47:52.083876: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 6, Chunks in use: 6. 3.0KiB allocated for chunks. 3.0KiB in use in bin. 3.0KiB client-requested in use in bin.
2024-09-15 14:47:

ResourceExhaustedError: Exception encountered when calling layer 'query' (type EinsumDense).

{{function_node __wrapped__Einsum_N_2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[7632000,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Einsum] name: 

Call arguments received by layer 'query' (type EinsumDense):
  • inputs=tf.Tensor(shape=(212000, 36, 1), dtype=float32)